# Introduction
This purpose of this notebook is to reproduce the regressions described in *The carbon footprint of household energy use in the United States* by Goldstein, Gounaridis, and Newell [PNAS](https://www.pnas.org/doi/abs/10.1073/pnas.1922205117).  The regressions used in that paper are described in more detail in separately-provided [Appendix](https://www.pnas.org/doi/suppl/10.1073/pnas.1922205117/suppl_file/pnas.1922205117.sapp.pdf).


## Table of Contents:
* [Data](#data)
* [Configuration](#config)
* [Checks](#checks)
* [Filter To Fields For Regressions](#preprocessing)
* [Profiling/Inspection](#profiling)
* [Miscellaneous Electricity](#elmisc)
* [Electricity For Space Heating](#elsph)
* [Electricity For Space Cooling](#elcol)
* [Electricity For Water Heating](#elwth)
* [Natural Gas Space Heating](#ngsph)
* [Natural Gas Water Heating](#ngwth)
* [Fuel Oil Space Heating](#fosph)
* [Fuel Oil Water Heating](#fowth)
* [Propane Space Heating](#lpsph)
* [Propane Water Heating](#lpwth)

# Data <a class="anchor" id="data"></a>
The regression are performed using the EIA's Residential Energy Consumption Survey (RECS) data.  For the 2015 survey, we use the [microdata data set](https://www.eia.gov/consumption/residential/data/2015/csv/recs2015_public_v4.csv).  To set up the regressions below, the [data dictionary](https://www.eia.gov/consumption/residential/data/2015/xls/codebook_publicv4.xlsx) proved very useful. 

While the logic below does support use of the 2020 survey results, the focus is on the 2015 data as that is what is used in the referenced paper.

# Imports

In [ ]:
import pandas as pd 
import numpy as np
import statsmodels.api as sm

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration <a class="anchor" id="config"></a>

In [ ]:
vintage = 2015

In [ ]:
water_heating_uses_fuelheat = True # True allows us to reproduce the water heating equations

In [ ]:
semiattached_codes = [3]
apartment_codes = [4, 5]

In [ ]:
write_energy_consumption_md = False

In [ ]:
df = pd.read_csv("./recs2015_public_v4.csv") if vintage == 2015 else pd.read_csv("./recs2020_public_v7.csv")

In [ ]:
elmisc_fields = [
    "KWHRFG",
    # Electricity usage for all refrigerators, in kilowatthours, 2015
    # KWHRFG1 Electricity usage for first refrigerators, in kilowatthours, 2015
    # KWHRFG2 Electricity usage for second refrigerators, in kilowatthours, 2015
    "KWHFRZ", # Electricity usage for freezers, in kilowatthours, 2015
    "KWHCOK", # Electricity usage for cooking (stoves, cooktops, and ovens), in kilowatthours, 2015
    "KWHMICRO", # Electricity usage for microwaves, in kilowatthours, 2015
    "KWHCW", # Electricity usage for clothes washers, in kilowatthours, 2015
    "KWHCDR", # Electricity usage for clothes dryers, in kilowatthours, 2015
    "KWHDWH", # Electricity usage for dishwashers, in kilowatthours, 2015
    "KWHLGT", # Electricity usage for indoor and outdoor lighting, in kilowatthours, 2015
    "KWHTVREL",
    # Electricity usage for all televisions and related peripherals, in kilowatthours, 2015
    # "KWHTV1",  usage for first televisions, in kilowatthours, 2015
    # "KWHTV2", Electricity usage for second televisions, in kilowatthours, 2015
    "KWHAHUHEAT", # Electricity usage for air handlers and boiler pumps used for heating, in kilowatthours, 2015
    "KWHAHUCOL", # Electricity usage for air handlers used for cooling, in kilowatthours, 2015
    "KWHEVAPCOL", # Electricity usage for evaporative coolers, in kilowatthours, 2015
    "KWHCFAN", # Electricity usage for ceiling fans, in kilowatthours, 2015
    "KWHDHUM", # Electricity usage for dehumidifiers, in kilowatthours, 2015
    "KWHHUM", # Electricity usage for humidifiers, in kilowatthours, 2015
    "KWHPLPMP", # Electricity usage for swimming pool pumps, in kilowatthours, 2015
    "KWHHTBPMP", # Electricity usage for hot tub pumps, in kilowatthours, 2015
    "KWHHTBHEAT", # Electricity usage for hot tub heaters, in kilowatthours, 2015
    "KWHNEC", # Electricity usage for other devices and purposes not elsewhere classified, in kilowatthours, 2015
] if vintage == 2015 else [
    "KWHRFG",
    "KWHFRZ",
    "KWHCOK",
    "KWHMICRO",
    "KWHCW",
    "KWHCDR",
    "KWHDWH",
    "KWHLGT",
    "KWHTVREL",
    "KWHAHUHEAT",
    "KWHAHUCOL",
    "KWHCFAN", #missing KWHEVAPCOL
    "KWHDHUM",
    "KWHHUM",
    "KWHPLPMP",
    "KWHHTBPMP",
    "KWHHTBHEAT",
    "KWHEVCHRG", #new for 2020
    "KWHNEC",
] 

df["KWHMISC"] = df[elmisc_fields].apply(np.sum, axis=1)

# if vintage == 2015:
#     df["KWHOTH"] = df.KWH - df.KWHSPH - df.KWHCOL - df.KWHWTH - df.KWHRFG

In [ ]:
pricespecs = [
    ("PRCEL", "DOLLAREL", "KWH"),
    ("PRCNG", "DOLLARNG", "CUFEETNG"),
    ("PRCLP", "DOLLARLP", "GALLONLP"),
    ("PRCFO", "DOLLARFO", "GALLONFO"),
] + [
    (f"PRCEL{heatingtype}", f"DOLEL{heatingtype}", f"KWH{heatingtype}") for heatingtype in ["SPH", "COL", "WTH"]
] + [
    (f"PRCNG{heatingtype}", f"DOLNG{heatingtype}", f"CUFEETNG{heatingtype}") for heatingtype in ["SPH", "WTH"]
] + [
    (f"PRCLP{heatingtype}", f"DOLLP{heatingtype}", f"GALLONLP{heatingtype}") for heatingtype in ["SPH", "WTH"]
] + [
    (f"PRCFO{heatingtype}", f"DOLFO{heatingtype}", f"GALLONFO{heatingtype}") for heatingtype in ["SPH", "WTH"]
]

def lambda_builder(dol, con): return lambda df0: np.where(df0[con] > 0.0, df0[dol] / df0[con], 0.0)

df = df.assign(
    **{ prc: lambda_builder(dol, con) for prc, dol, con in pricespecs }
)

# Checks <a class="anchor" id="checks"></a>

In [ ]:
# A check
df["KWHDIFF"] = (df.KWH - df.KWHSPH - df.KWHCOL - df.KWHWTH - df.KWHMISC).abs()
df.sort_values(["KWHDIFF"], ascending=False) [["KWH", "KWHSPH", "KWHCOL", "KWHWTH", "KWHMISC", "KWHDIFF"] + elmisc_fields].head(5).style

In [ ]:
(df.KWH - df.KWHSPH - df.KWHCOL - df.KWHWTH - df.KWHMISC).abs().max()

In [ ]:
# Check KWHOTH in 2020 data
if vintage == 2020:
    (df.KWH - df.KWHSPH -df.KWHCOL- df.KWHWTH - df.KWHRFG - df.KWHOTH).abs().max()


# Filter To Fields For Regressions <a class="anchor" id="preprocessing"></a>
We include some fields beyond those specified in the paper for experimentation.

In [ ]:
extra_weather_fields = ["HDD30YR", "CDD30YR"] if vintage == 2015 else ["HDD30YR_PUB", "CDDJOYR_PUB"]
extra_ac_fields = ["COOLTYPE", "CENACHP"] if vintage == 2015 else ["ACEQUIPM_PUB"]
extra_geo_fields = ["CLIMATE_REGION_PUB", "IECC_CLIMATE_PUB"] if vintage == 2015 else ["BA_climate", "IECC_climate_code"]

fields = [
    "REGIONC", "DIVISION",
    "HDD65",
    "CDD65"
] + extra_weather_fields + [
    "UATYP10",
    "TOTCSQFT", "TOTHSQFT", "TOTSQFT_EN",
    "YEARMADERANGE",
    "TYPEHUQ",
    "NHSLDMEM",
    "HEATHOME", "EQUIPM", "FUELHEAT", # space heating
    "AIRCOND"
] + extra_ac_fields + [
    "FUELH2O", # water heating fuel 
    "USEEL", "ELWARM", "ELCOOL", "ELWATER", 
    "USENG", "UGWARM", "UGWATER", 
    "USELP", "LPWARM", "LPWATER", 
    "USEFO", "FOWARM", "FOWATER", 
    "USEWOOD", "USESOLAR", "SOLWATER",
] + extra_geo_fields + [
    "KWH", "KWHSPH", "KWHCOL", "KWHWTH",
    "KWHMISC",
    "DOLLAREL", "DOLELSPH", "DOLELCOL", "DOLELWTH",
    "CUFEETNG", "CUFEETNGSPH", "CUFEETNGWTH", "BTUNG", "BTUNGSPH", "BTUNGWTH", # Check which BTU fields we want
    "DOLLARNG", "DOLNGSPH", "DOLNGWTH",
    "GALLONLP", "GALLONLPSPH", "GALLONLPWTH", "BTULP", "BTULPSPH", "BTULPWTH", #Check which BTU fields we want
    "DOLLARLP", "DOLLPSPH", "DOLLPWTH",
    "GALLONFO", "GALLONFOSPH", "GALLONFOWTH", "BTUFO", "BTUFOSPH", "BTUFOWTH", # Check which BTU fields we want 
    "DOLLARFO", "DOLFOSPH", "DOLFOWTH",
    "ATHOME", "EMPLOYHH", "NWEIGHT" # Adding for diagnostic purposes 
] + [fld for fld, _, _ in pricespecs]

df = df[fields]

if vintage == 2020:
    # the REGIONC field in 2020 data is a string label rather than a numeric code in the 2015 data, 
    # so we adjust here to avoid changing indicator variable definitions below.
    regionmap = {
        "NORTHEAST": 1,
        "MIDWEST": 2,
        "SOUTH": 3,
        "WEST": 4
    }
    df = df.assign(
        REGIONC2020 = lambda df0: df0.REGIONC,
        REGIONC = lambda df0: df0.REGIONC2020.map(regionmap)
    )


# Profiling/Inspection <a class="anchor" id="profiling"></a>

In [ ]:
# This can be compared against the histogram provided in the supplemental materials
# This is the floor area in m^2 for SF detached houses.
plt.hist(df.TOTSQFT_EN[df.TYPEHUQ== 2] / 10.7639, bins=15, weights=df.NWEIGHT[df.TYPEHUQ == 2], density=True)
plt.title("Single-Family Detached Household Size")
plt.xlabel("Household Size (square meters)")
plt.ylabel("Density")
# plt.savefig("sf_detached_household_size.png")
plt.show()

In [ ]:
# This can be compared against the histogram provided in the supplemental materials
# This is the floor area in m^2 for SF detached houses.
plt.hist(df.TOTSQFT_EN[df.TYPEHUQ.isin([4, 5])] / 10.7639, bins=13, weights=df.NWEIGHT[df.TYPEHUQ.isin([4, 5])], density=True)
plt.title("Apartment Size")
plt.xlabel("Household Size (square meters)")
# plt.savefig("apartment_household_size.png")
plt.ylabel("Density")
plt.show()

In [ ]:
#This is the floor area in m^2 for all properties.
plt.hist(df.TOTSQFT_EN / 10.7639, bins=15, weights=df.NWEIGHT)
plt.show()

In [ ]:
area_mean = (df.NWEIGHT[df.TYPEHUQ == 2] * (df.TOTSQFT_EN[df.TYPEHUQ == 2] / 10.7639)).sum()/df.NWEIGHT[df.TYPEHUQ == 2].sum()
print(f"Mean area (m^2) for single-family detached house: {area_mean}") 
print(f"Count for single-family detached house: {(df.TOTSQFT_EN[df.TYPEHUQ == 2] / 10.7639).count()}")

In [ ]:
area_mean = (df.NWEIGHT[df.TYPEHUQ.isin([4, 5])] * (df.TOTSQFT_EN[df.TYPEHUQ.isin([4, 5])] / 10.7639)).sum() / df.NWEIGHT[df.TYPEHUQ.isin([4, 5])].sum()
print(f"Mean area (m^2) for apartment: {area_mean}")

In [ ]:
area_mean = (df.NWEIGHT[df.TYPEHUQ.isin([5])] * (df.TOTSQFT_EN[df.TYPEHUQ.isin([5])] / 10.7639)).sum()/df.NWEIGHT[df.TYPEHUQ.isin([5])].sum()
print(f"Mean area (m^2) for apartment (buildings with 5 or more units): {area_mean}")

In [ ]:
area_mean = (df.NWEIGHT[df.TYPEHUQ.isin([3])] * (df.TOTSQFT_EN[df.TYPEHUQ.isin([3])] / 10.7639)).sum() / df.NWEIGHT[df.TYPEHUQ.isin([3])].sum()
print(f"Mean area (m^2) for SF attached house: {area_mean}")

In [ ]:
area_mean = (df.NWEIGHT[df.TYPEHUQ.isin([1])] * (df.TOTSQFT_EN[df.TYPEHUQ.isin([1])] / 10.7639)).sum() / df.NWEIGHT[df.TYPEHUQ.isin([1])].sum()
print(f"Mean area (m^2) for mobile home: {area_mean}") 

In [ ]:
# The binning isn't great here, but should be sufficient for basic profiling 
plt.hist(df.ATHOME, bins=6, density=True, weights=df.NWEIGHT)
plt.show()

# Miscellaneous Electricity <a class="anchor" id="elmisc"></a>
(1) QEL,misc = exp(β0 + β1ln(A) + β2Type + ẞ3Prel,i + ẞ4Statusj + B5YrRange + ẞ6log(n) + ẞ7NE)


In [ ]:
elmsc = df[(df.USEEL != 0)].assign(
    lnqelmisc = lambda df0: np.log(df0.KWHMISC),
    nweight = lambda df0: df0.NWEIGHT,
    lna = lambda df0: np.log(df0.TOTSQFT_EN),
    mobileind = lambda df0: (df0.TYPEHUQ == 1).astype(float),
    semiind = lambda df0: df0.TYPEHUQ.isin(semiattached_codes).astype(float),
    singleind = lambda df0: (df0.TYPEHUQ == 2).astype(float),
    prcelmsc = lambda df0: df0.PRCEL * 100.0, # cents per kWh
    yr = lambda df0: df0.YEARMADERANGE,
    lnn = lambda df0: np.log(df0.NHSLDMEM),
    isurban = lambda df0: df0.UATYP10.isin(["U", "C"]).astype(float),
    isnortheast = lambda df0: (df0.REGIONC == 1).astype(float),
)[[
    "lnqelmisc", "nweight",
    "lna",
    "mobileind", "semiind", "singleind",
    "prcelmsc",
    "yr",
    "lnn",
    "isurban",
    "isnortheast",
]]    
elmsc.describe()

In [ ]:
ytrain_elmsc = elmsc.lnqelmisc
weights = elmsc.nweight
xtrain_elmsc = sm.add_constant(elmsc.drop(columns=["lnqelmisc", "nweight"]))
model_estimator = sm.WLS(ytrain_elmsc, xtrain_elmsc, weights)

In [ ]:
model = model_estimator.fit()

In [ ]:
print(model.summary())

In [ ]:
# This for ease of copying to a spreadsheet
pd.DataFrame({
    "Estimate": model.params,
    "Std. Error": model.bse,
    "t value": model.tvalues,
    "P(>|t|)": model.pvalues
}).style

In [ ]:
if write_energy_consumption_md:
    pd.DataFrame({
        "Estimate": model.params,
        "Std. Error": model.bse,
        "t value": model.tvalues,
        "P(>|t|)": model.pvalues
    }).rename_axis(["Coefficients"], axis="index").to_markdown(
        "./EnergyConsumptionRegressions-elmisc.md",
        "a",
        index=True
    )

# Electricity For Space Heating <a class="anchor" id="elsph"></a>
(2) QEL,heat = exp(ẞ0 + β1ln(A) + B2Type + B3Prel,i + B4Yr + ẞ5In(HDD65,k) + ẞ6ln(CDD65,k) + B7S + ẞ8Statusj)

In [ ]:
# Note: For 2020 we must require KWHSPH > 0 as there appear to be some Os mixed in
# Note: For 2020 tried to add the following conditions to improve fits, to no avail: (df.KWHSPH > 200.0) & (df. TOTHSQFT > 400)
fuelcond = (df.HDD65 > 0) & (df.CDD65 > 0) & (df.FUELHEAT == 5)
if vintage == 2020:
    fuelcond &= (df.KWHSPH > 0)

elsph = df[fuelcond].assign(
    lnqelheat = lambda df0: np.log(df0.KWHSPH),
    nweight = lambda df0: df0.NWEIGHT,
    lna = lambda df0: np.log(df0.TOTSQFT_EN),
    mobileind = lambda df0: (df0.TYPEHUQ == 1).astype(float),
    semiind = lambda df0: df0.TYPEHUQ.isin(semiattached_codes).astype(float),
    singleind = lambda df0: (df0.TYPEHUQ == 2).astype(float),
    prcelsph = lambda df0: df0.PRCELSPH * 100.0, # cents per kWh
    yr = lambda df0: df0.YEARMADERANGE,
    lnhdd = lambda df0: np.log(df0.HDD65),
    lncdd = lambda df0: np.log(df0.CDD65),
    issouth = lambda df0: (df0.REGIONC == 3).astype (float),
    isnortheast = lambda df0: (df0.REGIONC == 1).astype(float),
    ismidwest = lambda df0: (df0.REGIONC == 2).astype(float),
    isurban = lambda df0: df0.UATYP10.isin(["U", "C"]).astype (float),
)[[
    "lnqelheat", "nweight",
    "lna", "mobileind", "semiind", "singleind", 
    "prcelsph", "yr", 
    "lnhdd", "lncdd",
    "issouth",
    "isurban"
]] 
elsph.describe()

In [ ]:
ytrain_elsph = elsph.lnqelheat
weights = elsph.nweight
xtrain_elsph = sm.add_constant(elsph.drop(columns=["lnqelheat", "nweight"]))
model_estimator = sm.WLS(ytrain_elsph, xtrain_elsph, weights)

In [ ]:
model = model_estimator.fit()

In [ ]:
print(model.summary())

In [ ]:
pd.DataFrame({
    "Estimate": model.params,
    "Std. Error": model.bse,
    "t value": model.tvalues,
    "P(>|t|)": model.pvalues
}).style

In [ ]:
if write_energy_consumption_md:
    pd.DataFrame({
        "Estimate": model.params,
        "Std. Error": model.bse,
        "t value": model.tvalues,
        "P(>|t|)": model.pvalues
    }).rename_axis(["Coefficients"], axis="index").to_markdown(
        "./EnergyConsumptionRegressions-elsph.md",
        "a",
        index=True
    )

# Electricity For Space Cooling <a class="anchor" id="elcol"></a>
(3) QEL,cool = exp(ẞ0 + β1ln(A) + β2Type + ẞ3Prel,i + ẞ4Yr + ẞ5In(CDD65,k) + β6S)

In [ ]:
elcol = df[(df.ELCOOL != 0) & (df.KWHCOL > 0) & (df.CDD65 > 0)].assign(
    lnqelcol = lambda df0: np.log(df0.KWHCOL),
    nweight = lambda df0: df0.NWEIGHT,
    lna = lambda df0: np.log(df0.TOTSQFT_EN),
    mobileind = lambda df0: (df0.TYPEHUQ == 1).astype(float),
    semiind = lambda df0: df0.TYPEHUQ.isin(semiattached_codes).astype(float),
    singleind = lambda df0: (df0.TYPEHUQ == 2).astype(float),
    prcelcol = lambda df0: df0.PRCELCOL * 100.10, # cents per kWh
    yr = lambda df0: df0.YEARMADERANGE,
    lncdd = lambda df0: np.log(df0.CDD65),
    issouth = lambda df0: (df0.REGIONC == 3).astype(float),
    isnortheast = lambda df0: (df0.REGIONC == 1).astype(float),
    ismidwest = lambda df0: (df0.REGIONC == 2).astype(float),
)[[
    "lnqelcol", "nweight",
    "lna", 
    "mobileind", "semiind", "singleind", 
    "prcelcol", "yr",
    "lncdd",
    "issouth",
]]
elcol.describe()

In [ ]:
ytrain_elcol = elcol.lnqelcol
weights = elcol.nweight
xtrain_elcol = sm.add_constant(elcol.drop(columns=["lnqelcol", "nweight"]))
model_estimator = sm.WLS(ytrain_elcol, xtrain_elcol, weights)

In [ ]:
model = model_estimator.fit()

In [ ]:
print(model.summary())

In [ ]:
pd.DataFrame({
    "Estimate": model.params,
    "Std. Error": model.bse,
    "t value": model.tvalues,
    "P(>|t|)": model.pvalues
}).style

In [ ]:
if write_energy_consumption_md:
    pd.DataFrame({
        "Estimate": model.params,
        "Std. Error": model.bse,
        "t value": model.tvalues,
        "P(>|t|)": model.pvalues
    }).rename_axis(["Coefficients"], axis="index").to_markdown(
        "./EnergyConsumptionRegressions-elcol.md",
        "a",
        index=True
    )

# Electricity For Water Heating <a class="anchor" id="elwth"></a>
(4) QEL,water = exp(ẞ0 + B1Prel,i + B2Yr + B3HDD65,k + ẞ4ln(n) + ẞ5Region + ẞ6Statusj)

In [ ]:
fuelcond = (df.FUELH2O == 5) & (df.KWHWTH > 0)
if water_heating_uses_fuelheat:
    fuelcond = (df.FUELHEAT == 5) & (df.KWHWTH > 0)

In [ ]:
elwth = df[fuelcond].assign(
    lnqelwth = lambda df0: np.log(df0.KWHWTH),
    nweight = lambda df0: df0.NWEIGHT,
    lnn = lambda df0: np.log(df0.NHSLDMEM),
    prcelwth = lambda df0: df0.PRCELWTH * 100.0, # cents per kWh
    yr = lambda df0: df0.YEARMADERANGE,
    hdd = lambda df0: df0.HDD65,
    issouth = lambda df0: (df0.REGIONC == 3).astype(float),
    isnortheast = lambda df0: (df0.REGIONC == 1).astype(float),
    iswest = lambda df0: (df0.REGIONC == 4).astype (float),
    isurban = lambda df0: df0.UATYP10.isin(["U", "C"]).astype(float),
)[[
    "lnqelwth", "nweight",
    "prcelwth",
    "yr",
    "hdd",
    "lnn",
    "isnortheast", "issouth", "iswest",
    "isurban"
]]
elwth.describe()

In [ ]:
ytrain_elwth = elwth.lnqelwth
weights = elwth.nweight
xtrain_elwth = sm.add_constant(elwth.drop(columns=["lnqelwth", "nweight"]))
model_estimator = sm.WLS(ytrain_elwth, xtrain_elwth, weights)

In [ ]:
model = model_estimator.fit()

In [ ]:
print(model.summary())

In [ ]:
pd.DataFrame({
    "Estimate": model.params,
    "Std. Error": model.bse,
    "t value": model.tvalues,
    "P(>|t|)": model.pvalues
}).style

In [ ]:
if write_energy_consumption_md:
    pd.DataFrame({
        "Estimate": model.params,
        "Std. Error": model.bse,
        "t value": model.tvalues,
        "P(>|t|)": model.pvalues
    }).rename_axis(["Coefficients"], axis="index").to_markdown(
        "./EnergyConsumptionRegressions-elwth.md",
        "a",
        index=True
    )

# Natural Gas Space Heating <a class="anchor" id="ngsph"></a>
(5) QNG,heat = exp(ẞ0 + β1ln(A) + β2Prng,i + ẞ3Yr + ẞ4ln(HDD65,k) + ẞ5Region + ẞ6Type)

In [ ]:
# Note: For 2020, adding (df.PRCNGSPH < 2) to filter only leads to small improvement

In [ ]:
ngsph = df[(df.HDD65 > 0) & (df.FUELHEAT == 1)].assign(
    lnqngheat = lambda df0: np.log(df0.BTUNGSPH),
    nweight = lambda df0: df0.NWEIGHT,
    lna = lambda df0: np.log(df0.TOTSQFT_EN),
    mobileind = lambda df0: (df0.TYPEHUQ == 1).astype (float),
    semiind = lambda df0: df0.TYPEHUQ.isin(semiattached_codes).astype(float),
    singleind = lambda df0: (df0.TYPEHUQ == 2).astype(float),
    prcngsph = lambda df0: 10.0* df0.PRCNGSPH, # dollars per 100 cuft converted to dollars per 1000 cuft
    yr = lambda df0: df0.YEARMADERANGE,
    lnhdd = lambda df0: np.log(df0.HDD65),
    issouth = lambda df0: (df0.REGIONC == 3).astype(float),
    isnortheast = lambda df0: (df0.REGIONC == 1).astype(float),
    iswest = lambda df0: (df0.REGIONC == 4).astype(float),
)[[
    "lnqngheat", "nweight",
    "lna",
    "prcngsph",
    "yr",
    "lnhdd",
    "isnortheast", "issouth", "iswest",
    "mobileind", "semiind", "singleind",
]]
ngsph.describe()

In [ ]:
ytrain_ngsph = ngsph.lnqngheat
weights = ngsph.nweight
xtrain_ngsph = sm.add_constant(ngsph.drop(columns=["lnqngheat", "nweight"]))
model_estimator = sm.WLS(ytrain_ngsph, xtrain_ngsph, weights)

In [ ]:
model = model_estimator.fit()

In [ ]:
print(model.summary())

In [ ]:
pd.DataFrame({
    "Estimate": model.params,
    "Std. Error": model.bse,
    "t value": model.tvalues,
    "P(>|t|)": model.pvalues
}).style


In [ ]:
if write_energy_consumption_md:
    pd.DataFrame({
        "Estimate": model.params,
        "Std. Error": model.bse,
        "t value": model.tvalues,
        "P(>|t|)": model.pvalues
    }).rename_axis(["Coefficients"], axis="index").to_markdown(
        "./EnergyConsumptionRegressions-ngsph.md",
        "a",
        index=True
    )

# Natural Gas Water Heating <a class="anchor" id="ngwth"></a>
(6) QNG,heat = exp(ẞ0 + ẞ1ln(n) + ẞ2Prng,i + B3Yr + ẞ4In(HDD65,k) + B5Region)


In [ ]:
df[["UGWATER", "FUELH2O"]].drop_duplicates().sort_values(["UGWATER"])

In [ ]:
fuelcond = (df.FUELH2O == 1) & (df.BTUNGWTH > 0)
if water_heating_uses_fuelheat:
    fuelcond = (df.FUELHEAT == 1) & (df.BTUNGWTH > 0)

In [ ]:
ngwth = df[fuelcond].assign(
    lnqngwth = lambda df0: np.log(df0.BTUNGWTH),
    nweight = lambda df0: df0.NWEIGHT,
    lnn = lambda df0: np.log(df0.NHSLDMEM),
    prcngwth = lambda df0: 10.0 * df0.PRCNGWTH, # dollars per 100 cuft converted to dollars per 1000 cuft 
    yr = lambda df0: df0.YEARMADERANGE,
    hdd = lambda df0: df0.HDD65,
    isnortheast = lambda df0: (df0.REGIONC == 1).astype(float), 
    issouth = lambda df0: (df0.REGIONC == 3).astype(float),
    iswest = lambda df0: (df0.REGIONC == 4).astype(float),
)[[
    "lnqngwth", "nweight",
    "lnn",
    "prcngwth",
    "yr",
    "hdd",
    "isnortheast", "issouth", "iswest",
]]
ngwth.describe()

In [ ]:
ytrain_ngwth = ngwth.lnqngwth
weights = ngwth.nweight
xtrain_ngwth = sm.add_constant(ngwth.drop(columns=["lnqngwth", "nweight"]))
model_estimator = sm.WLS(ytrain_ngwth, xtrain_ngwth, weights)

In [ ]:
model = model_estimator.fit()

In [ ]:
print(model.summary())

In [ ]:
pd.DataFrame({
    "Estimate": model.params,
    "Std. Error": model.bse,
    "t value": model.tvalues,
    "P(>|t|)": model.pvalues
}).style

In [ ]:
if write_energy_consumption_md:
    pd.DataFrame({
        "Estimate": model.params,
        "Std. Error": model.bse,
        "t value": model.tvalues,
        "P(>|t|)": model.pvalues
    }).rename_axis(["Coefficients"], axis="index").to_markdown(
        "./EnergyConsumptionRegressions-ngwth.md",
        "a",
        index=True
    )

# Fuel Oil Space Heating <a class="anchor" id="fosph"></a>
(9) QFO,heat = exp(β0 + β1ln(A) + B2Yr + B3Prfo,i + B4HDD65,k + ẞ5Apt)

In [ ]:
fosph = df[(df.FUELHEAT == 3)].assign(
    lnqfoheat = lambda df0: np.log(df0.BTUFOSPH),
    nweight = lambda df0: df0.NWEIGHT,
    lna = lambda df0: np.log(df0.TOTSQFT_EN),
    yr = lambda df0: df0.YEARMADERANGE,
    prcfosph = lambda df0: df0.PRCFOSPH,
    hdd = lambda df0: df0.HDD65,
    aptind = lambda df0: (df0.TYPEHUQ.isin(apartment_codes)).astype(float),
)[[
    "lnqfoheat", "nweight",
    "lna",
    "yr",
    "prcfosph",
    "hdd",
    "aptind",
]]
fosph.describe()

In [ ]:
ytrain_fosph = fosph.lnqfoheat
weights = fosph.nweight
xtrain_fosph = sm.add_constant(fosph.drop(columns=["lnqfoheat", "nweight"]))
model_estimator = sm.WLS(ytrain_fosph, xtrain_fosph, weights)
model = model_estimator.fit()

In [ ]:
print(model.summary())

In [ ]:
pd.DataFrame({
    "Estimate": model.params,
    "Std. Error": model.bse,
    "t value": model.tvalues,
    "P(>|t|)": model.pvalues
}).style

In [ ]:
if write_energy_consumption_md:
    pd.DataFrame({
        "Estimate": model.params,
        "Std. Error": model.bse,
        "t value": model.tvalues,
        "P(>|t|)": model.pvalues
    }).rename_axis(["Coefficients"], axis="index").to_markdown(
        "./EnergyConsumptionRegressions-fosph.md",
        "a",
        index=True
    )

In [ ]:
plt.scatter(np.log(fosph.hdd), fosph.lnqfoheat, alpha=0.5)
plt.show()

# Fuel Oil Water Heating <a class="anchor" id="fowth"></a>
(10) QNG,heat = exp(30 + ẞ1ln(n) + ẞ2Prfo,i + B3W)

In [ ]:
df.groupby(["FOWATER", "FUELHEAT", "FUELH2O"]).agg(ct=("FOWATER", "count")).loc[1].reset_index()

In [ ]:
fuelcond = (df.FUELH2O == 3) 
if water_heating_uses_fuelheat: 
    fuelcond = (df.FUELHEAT == 3) & (df.BTUFOWTH > 0)

In [ ]:
fowth = df[fuelcond].assign(
    lnqfowth = lambda df0: np.log(df0.BTUFOWTH),
    nweight = lambda df0: df0.NWEIGHT,
    lnn = lambda df0: np.log(df0.NHSLDMEM),
    prcfowth = lambda df0: df0.PRCFOWTH, # dollars per gallon
    iswest = lambda df0: (df0.REGIONC == 4).astype (float),
)[[
    "lnqfowth", "nweight",
    "lnn",
    "prcfowth",
    "iswest",
]]
fowth.describe()

In [ ]:
ytrain_fowth = fowth.lnqfowth
weights = fowth.nweight
xtrain_fowth = sm.add_constant(fowth.drop(columns=["lnqfowth", "nweight"]))
model_estimator = sm.WLS(ytrain_fowth, xtrain_fowth, weights)
model = model_estimator.fit()

In [ ]:
print(model.summary())

In [ ]:
pd.DataFrame({
    "Estimate": model.params,
    "Std. Error": model.bse,
    "t value": model.tvalues,
    "P(>|t|)": model.pvalues
}).style


In [ ]:
if write_energy_consumption_md:
    pd.DataFrame({
        "Estimate": model.params,
        "Std. Error": model.bse,
        "t value": model.tvalues,
        "P(>|t|)": model.pvalues
    }).rename_axis(["Coefficients"], axis="index").to_markdown(
        "./EnergyConsumptionRegressions-ngwth.md",
        "a",
        index=True
    )

In [ ]:
plt.scatter(np.log(fowth.prcfowth), fowth.lnqfowth, alpha=0.5)
plt.show()


In [ ]:
sns.boxplot(x='iswest', y='lnqfowth', data=fowth)

# Propane Space Heating <a class="anchor" id="lpsph"></a>
(7) QLP,heat = exp(ẞ0 + β1ln(A) + β2HDD65,k + B3Yr + B6Type + B4W)

In [ ]:
df[["LPWARM", "FUELHEAT"]].drop_duplicates().sort_values (["LPWARM"])

In [ ]:
lpsph = df[(df.FUELHEAT == 2)].assign(
    lnqlpheat = lambda df0: np.log(df0.BTULPSPH),
    nweight = lambda df0: df0.NWEIGHT,
    lna = lambda df0: np.log(df0.TOTSQFT_EN),
    hdd = lambda df0: df0.HDD65,
    yr = lambda df0: df0.YEARMADERANGE,
    mobileind = lambda df0: (df0.TYPEHUQ == 1).astype(float),
    semiind = lambda df0: df0.TYPEHUQ.isin(semiattached_codes).astype(float),
    singleind = lambda df0: (df0. TYPEHUQ == 2).astype (float),
    iswest = (df.REGIONC == 4).astype(float),
)[[
    "lnqlpheat", "nweight",
    "lna",
    "hdd",
    "yr",
    "mobileind", "semiind", "singleind",
    "iswest"
]]
lpsph.describe()

In [ ]:
ytrain_lpsph = lpsph.lnqlpheat
weights = lpsph.nweight
xtrain_lpsph = sm.add_constant(lpsph.drop(columns=["lnqlpheat", "nweight"]))
model_estimator = sm.WLS(ytrain_lpsph, xtrain_lpsph, weights)

In [ ]:
model = model_estimator.fit()
print(model.summary())

In [ ]:
pd.DataFrame({
    "Estimate": model.params, 
    "Std. Error": model.bse,
    "t value": model.tvalues,
    "P(>|t|)": model.pvalues,
}).style

In [ ]:
if write_energy_consumption_md:
    pd.DataFrame({
        "Estimate": model.params,
        "Std. Error": model.bse,
        "t value": model.tvalues,
        "P(>|t|)": model.pvalues
    }).rename_axis(["Coefficients"], axis="index").to_markdown(
        "./EnergyConsumptionRegressions-lpsph.md",
        "a",
        index=True
    )

# Propane Water Heating <a class="anchor" id="lpwth"></a>
(8) QLP,heat = exp(ẞ0 + ẞ1ln(n) + β2ln(HDD65,k) + ẞ3Status)


In [ ]:
df[["LPWATER", "FUELH2O"]].drop_duplicates().sort_values(["LPWATER"])

In [ ]:
df.groupby(["LPWATER", "FUELHEAT", "FUELH2O"]).agg(ct=("LPWATER", "count")).loc[1].reset_index() # Just Look at LPWATER == 1

In [ ]:
fuelcond = (df.FUELH2O == 2)
if water_heating_uses_fuelheat: 
    fuelcond = (df.FUELHEAT == 2) & (df.BTULPWTH > 0)

In [ ]:
lpwth = df[fuelcond].assign(
    lnqlpwth = lambda df0: np.log(df0.BTULPWTH),
    nweight = lambda df0: df0.NWEIGHT,
    lnn = lambda df0: np.log(df0.NHSLDMEM),
    hdd = lambda df0: df0.HDD65,
    isurban = lambda df0: df.UATYP10.isin(["U", "C"]).astype(float),
)[[
    "lnqlpwth", "nweight",
    "lnn",
    "hdd",
    "isurban",
]]
lpwth.describe()

In [ ]:
ytrain_lpwth = lpwth.lnqlpwth
weights = lpwth.nweight
xtrain_lpwth = sm.add_constant(lpwth.drop(columns=["lnqlpwth", "nweight"]))
model_estimator = sm.WLS(ytrain_lpwth, xtrain_lpwth, weights)
model = model_estimator.fit()

In [ ]:
print(model.summary())

In [ ]:
pd.DataFrame({
    "Estimate": model.params,
    "Std. Error": model.bse,
    "t value": model.tvalues,
    "P(>|t|)": model.pvalues
}).style

In [ ]:
if write_energy_consumption_md:
    pd.DataFrame({
        "Estimate": model.params,
        "Std. Error": model.bse,
        "t value": model.tvalues,
        "P(>|t|)": model.pvalues
    }).rename_axis(["Coefficients"], axis="index").to_markdown(
        "./EnergyConsumptionRegressions-lpwth.md",
        "a",
        index=True
    )

In [ ]:
plt.scatter(np.log(lpwth.lnn), lpwth.lnqlpwth, alpha=0.5)
plt.show()


In [ ]:
plt.scatter(lpwth.hdd, lpwth.lnqlpwth, alpha=0.5)
plt.show()

In [ ]:
import seaborn as sns
sns.boxplot(x='isurban', y='lnqlpwth', data=lpwth)